In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd 
import os
import time 

In [30]:
def get_premgames (code, player):
    base_url = f'https://fbref.com/en/players/{code}/{player}'
    html = requests.get(base_url).text
    soup = BeautifulSoup(html, 'lxml')
    summary = soup.find('table', class_='stats_table sortable min_width')
    
    comp = summary.find_all('td', attrs={'data-stat': 'comp_level'})
    comp_text = [cell.get_text() for cell in comp]
    games = summary.find_all('td', attrs={'data-stat': 'games'})
    games_text = [cell.get_text() for cell in games]
    
    df = pd.DataFrame({
        'Competition': comp_text, 
        'Games Played': games_text
    })
    total_games = df[df['Competition'] == '1. Premier League']['Games Played']
    total_games = pd.to_numeric(total_games)
    prem_games = total_games.sum() - total_games.iloc[-1]
    return(prem_games)

In [48]:
base_url = 'https://fbref.com/en/players/3a24769f/Ross-Barkley'
html = requests.get(base_url).text
soup = BeautifulSoup(html, 'lxml')
summary = soup.find('table', class_='stats_table sortable min_width')


In [50]:
table = summary.find('tbody')

In [78]:
comp = table.find_all('td', attrs={'data-stat': 'comp_level'})
comp_text = [cell.get_text() for cell in comp]
games = table.find_all('td', attrs={'data-stat': 'games'})
games_text = [cell.get_text() for cell in games]
season = table.find_all('th', attrs={'data-stat': 'year_id'})
season_text  = [cell.get_text() for cell in season]
    
df = pd.DataFrame({
    'Season': season_text,
    'Competition': comp_text, 
    'Games Played': games_text
})

season_list = ('2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018')
total_games = df[(df['Competition'] == '1. Premier League') & (df['Season'].isin(season_list))]['Games Played']
total_games = pd.to_numeric(total_games)
prem_games = total_games.sum()

In [79]:
prem_games

114

In [63]:
df

,Season,Competition,Games Played
0,2011-2012,1. Premier League,6
1,2012-2013,2. Championship,4
2,2012-2013,2. Championship,13
3,2012-2013,1. Premier League,7
4,2013-2014,1. Premier League,34
5,2014-2015,1. Premier League,29
6,2015-2016,1. Premier League,38
7,2016-2017,1. Premier League,36
8,2017-2018,Jr. PL2 — Div. 1,1
9,2017-2018,1. Premier League,2


In [32]:
get_premgames('3a24769f', 'Ross-Barkley')

264

In [5]:
def get_url_final(code, year_range, category, player):
    base_url = 'https://fbref.com/en/players/{}/matchlogs/{}/c9/{}/{}-Match-Logs'
    url = base_url.format(code, year_range, category, player)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    html_filtered = soup.find('tbody')
    return(html_filtered)

In [4]:
def get_dat(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    return (coldat)   

def get_date(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('th', {'data-stat': col}).text)
    return (coldat)   


def get_matchweek(dat, col):
    coldat = []
    for row in dat.find_all('tr'):
        if not row.attrs:
            coldat.append(row.find('td', {'data-stat': col}).text)
    coldat = [coldat.replace('Matchweek ', '') for coldat in coldat]
    return(coldat)

In [188]:
data_summary = get_url_final('e6af02e0', '2022-2023', '', 'Matheus-Nunes')

In [191]:
data_passing = get_url_final('e6af02e0', '2022-2023', 'passing', 'Matheus-Nunes')

In [197]:
date = get_date(data_summary, 'date')
day = get_dat(data_summary,'dayofweek')
matchweek = get_matchweek(data_summary, 'round')
venue = get_dat(data_summary, 'venue')
result = get_dat(data_summary, 'result')
team = get_dat(data_summary, 'team')
opponent = get_dat(data_summary, 'opponent')
start = get_dat(data_summary, 'game_started')
position = get_dat(data_summary, 'position')
mins = get_dat(data_summary, 'minutes')
mins = [x if x != '' else 0 for x in mins]

goals = get_dat(data_summary, 'goals')
assist = get_dat(data_summary, 'assists')
pen_goals = get_dat(data_summary, 'pens_made')
pen_attempted = get_dat(data_summary, 'pens_att')
shots = get_dat(data_summary, 'shots')
sot = get_dat(data_summary, 'shots_on_target')
yellow = get_dat(data_summary, 'cards_yellow')
red = get_dat(data_summary, 'cards_red')
touches = get_dat(data_summary, 'touches')
tackles = get_dat(data_summary, 'tackles')
interceptions = get_dat(data_summary, 'interceptions')
blocks = get_dat(data_summary, 'blocks')
xg = get_dat(data_summary, 'xg')
npxg = get_dat(data_summary, 'npxg')
xag = get_dat(data_summary, 'xg_assist')
sca = get_dat(data_summary, 'sca')
gca = get_dat(data_summary, 'gca')
passes_completed = get_dat(data_summary, 'passes_completed')
passes_attempted = get_dat(data_summary, 'passes')
prg_passes = get_dat(data_summary, 'progressive_passes')
carries = get_dat(data_summary, 'carries')
prg_carries = get_dat(data_summary, 'progressive_carries')
takeon_att = get_dat(data_summary, 'take_ons')
takeon_suc = get_dat(data_summary, 'take_ons_won')


pass_distance = get_dat(data_passing, 'passes_total_distance')
prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')
short_pass_completed = get_dat(data_passing, 'passes_completed_short')
short_pass_attempted = get_dat(data_passing, 'passes_short')
med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
med_pass_attempted = get_dat(data_passing, 'passes_medium')
long_pass_completed = get_dat(data_passing, 'passes_completed_long')
long_pass_attempted = get_dat(data_passing, 'passes_long')
xa = get_dat(data_passing, 'pass_xa')
keypass = get_dat(data_passing, 'assisted_shots')
finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    

In [198]:
df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross
    
        # 'Live Pass': live_pass,
        # 'Dead Pass': dead_pass,
        # 'Free Kick Pass': freekick_pass,
        # 'Through Balls': through_balls,
        # 'Switches': switches,
        # 'Crosses': crosses,
        # 'Throw Ins Taken': throwin,
        # 'Corners Taken': corners,
        # 'Passes Offside': offside_pass,
    
        # 'Live SCA': live_sca,
        # 'Deadball SCA': deadball_sca,
        # 'Take-on SCA': takeons_sca,
        # 'Shot SCA': shots_sca,
        # 'Foul SCA': fouls_sca,
        # 'Defense SCA': defense_sca,
    
        # 'Live GCA': live_gca,
        # 'Deadball GCA': deadball_gca,
        # 'Take-on GCA': takeons_gca,
        # 'Shot GCA': shots_gca,
        # 'Foul GCA': fouls_gca,
        # 'Defense GCA': defense_gca,
    
        # 'Tackles Won': tackles_won,
        # 'Defensive Third Tackles': tackles_def,
        # 'Middle Third Tackles': tackles_mid,
        # 'Attacking Third Tackles': tackles_att,
        # 'Dribblers Tackled': dribblers_tackled,
        # 'Dribblers Tackled Attempts': dribbles_challenged,
        # 'Challenges Lost': challenges_lost,
        # 'Shots Blocked': shots_blocked,
        # 'Passes Blocked': pass_blocked,
        # 'Clearances': clearances,
        # 'Defensive Errors': def_errors,
    
        # 'Defensive Penalty Area Touches': def_penarea_touches,
        # 'Defensive Third Touches': def_third_touches,
        # 'Middle Third Touches': mid_third_touches,
        # 'Attacking Third Touches': att_third_touches,
        # 'Penalty Area Touches': pen_area_touches,
        # 'Carry Distance': carry_distance,
        # 'Progressive Carry Distance': prg_carry_distance,
        # 'Final Third Carries': final_third_carries,
        # 'Carries into Penalty Area': pen_area_carries,
        # 'Miscontrols': miscontrols,
        # 'Dispossessed': dispossesed,
        # 'Passes Received': pass_received,
        # 'Progressive Passes Received': prg_pass_received
    })


In [199]:
df = df.astype({
    'Date': 'datetime64[ns]',
    'Day': 'object',
    'Matchweek': 'int64',
    'Venue': 'object',
    'Result': 'object',
    'Team': 'object',
    'Opponent': 'object', 
    'Start': 'object', 
    'Position': 'object',
    'Minutes Played': 'int64', 
    'Goals': 'int64', 
    'Assists': 'int64', 
    'Penalties Scored': 'int64', 
    'Penalties Attempted': 'int64', 
    'Shots': 'int64', 
    'Shots on Target': 'int64', 
    'Yellow Cards': 'int64'
})

In [200]:
print(mins)

['90', '90', '74', '90', '90', '71', '90', '89', '57', '65', '40', '11', '32', '90', '90', '90', '90', '90', '69', '60', '75', '63', '66', '23', 0, '90', '89', '90', '80', '88', '90', '89', '90', '21', '45']


In [23]:
#now, we want to modifiy the function above, to allow us to get any players data that we want, not just salah

def get_data(code, year_range, player, fpl_name):
    data_summary = get_url_final(code, year_range, '', player)
    if not data_summary:
        empty = pd.DataFrame(columns = column_names)
        return(empty)
    
    data_passing = get_url_final(code, year_range, 'passing', player)
    data_passingtype = get_url_final(code, year_range, 'passing_types', player)
    data_gca = get_url_final(code, year_range, 'gca', player)
    data_defense = get_url_final(code, year_range, 'defense', player)
    data_possession = get_url_final(code, year_range, 'possession', player)
    
    ################################################################
    #getting the data for the necessary columns
    
    ##### summary data #####
    #misc data 
    date = get_date(data_summary, 'date')
    day = get_dat(data_summary,'dayofweek')
    matchweek = get_matchweek(data_summary, 'round')
    venue = get_dat(data_summary, 'venue')
    result = get_dat(data_summary, 'result')
    team = get_dat(data_summary, 'team')
    opponent = get_dat(data_summary, 'opponent')
    start = get_dat(data_summary, 'game_started')
    position = get_dat(data_summary, 'position')
    mins = get_dat(data_summary, 'minutes')
    
    #this line is to guard against dataframes where there are missing entries for mins played. for example, matheus nunes somehow had a red card with 
    #0 mins played. 
    mins = [x if x != '' else 0 for x in mins]

    #performance data
    goals = get_dat(data_summary, 'goals')
    assist = get_dat(data_summary, 'assists')
    pen_goals = get_dat(data_summary, 'pens_made')
    pen_attempted = get_dat(data_summary, 'pens_att')
    shots = get_dat(data_summary, 'shots')
    sot = get_dat(data_summary, 'shots_on_target')
    yellow = get_dat(data_summary, 'cards_yellow')
    red = get_dat(data_summary, 'cards_red')
    touches = get_dat(data_summary, 'touches')
    tackles = get_dat(data_summary, 'tackles')
    interceptions = get_dat(data_summary, 'interceptions')
    blocks = get_dat(data_summary, 'blocks')

    #expected performance metrics
    xg = get_dat(data_summary, 'xg')
    npxg = get_dat(data_summary, 'npxg')
    xag = get_dat(data_summary, 'xg_assist')

    #shot creating actions
    sca = get_dat(data_summary, 'sca')
    gca = get_dat(data_summary, 'gca')

    #passes
    passes_completed = get_dat(data_summary, 'passes_completed')
    passes_attempted = get_dat(data_summary, 'passes')
    prg_passes = get_dat(data_summary, 'progressive_passes')

    #carries 
    carries = get_dat(data_summary, 'carries')
    prg_carries = get_dat(data_summary, 'progressive_carries')

    #take-ons
    takeon_att = get_dat(data_summary, 'take_ons')
    takeon_suc = get_dat(data_summary, 'take_ons_won')
    
    ##### passing data #####
    #total
    pass_distance = get_dat(data_passing, 'passes_total_distance')
    prg_pass_distance = get_dat(data_passing, 'passes_progressive_distance')

    #short
    short_pass_completed = get_dat(data_passing, 'passes_completed_short')
    short_pass_attempted = get_dat(data_passing, 'passes_short')

    #medium 
    med_pass_completed = get_dat(data_passing, 'passes_completed_medium')
    med_pass_attempted = get_dat(data_passing, 'passes_medium')

    #long
    long_pass_completed = get_dat(data_passing, 'passes_completed_long')
    long_pass_attempted = get_dat(data_passing, 'passes_long')

    #misc
    xa = get_dat(data_passing, 'pass_xa')
    keypass = get_dat(data_passing, 'assisted_shots')
    finalthird_pass = get_dat(data_passing, 'passes_into_final_third')
    penaltyarea_pass = get_dat(data_passing, 'passes_into_penalty_area')
    penaltyarea_cross = get_dat(data_passing, 'crosses_into_penalty_area')
    
    
    ##### passing types data #####
    live_pass = get_dat(data_passingtype, 'passes_live')
    dead_pass = get_dat(data_passingtype, 'passes_dead')
    freekick_pass = get_dat(data_passingtype, 'passes_free_kicks')
    through_balls = get_dat(data_passingtype, 'through_balls')
    switches = get_dat(data_passingtype, 'passes_switches')
    crosses = get_dat(data_passingtype, 'crosses')
    throwin = get_dat(data_passingtype, 'throw_ins')
    corners = get_dat(data_passingtype, 'corner_kicks')
    offside_pass = get_dat(data_passingtype, 'passes_offsides')
    
    
    ##### gca data #####
    #sca types 
    live_sca = get_dat(data_gca, 'sca_passes_live')
    deadball_sca = get_dat(data_gca, 'sca_passes_dead')
    takeons_sca = get_dat(data_gca, 'sca_take_ons')
    shots_sca = get_dat(data_gca, 'sca_shots')
    fouls_sca = get_dat(data_gca, 'sca_fouled')
    defense_sca = get_dat(data_gca, 'sca_defense')

    #gca types 
    live_gca = get_dat(data_gca, 'gca_passes_live')
    deadball_gca = get_dat(data_gca, 'gca_passes_dead')
    takeons_gca = get_dat(data_gca, 'gca_take_ons')
    shots_gca = get_dat(data_gca, 'gca_shots')
    fouls_gca = get_dat(data_gca, 'gca_fouled')
    defense_gca = get_dat(data_gca, 'gca_defense')
    
    
    ##### defensive data ######
    #tackles
    tackles_won = get_dat(data_defense,'tackles_won')
    tackles_def = get_dat(data_defense, 'tackles_def_3rd')
    tackles_mid = get_dat(data_defense, 'tackles_mid_3rd')
    tackles_att = get_dat(data_defense, 'tackles_att_3rd')

    #challenges 
    dribblers_tackled = get_dat(data_defense, 'challenge_tackles')
    dribbles_challenged = get_dat(data_defense, 'challenges')
    challenges_lost = get_dat(data_defense, 'challenges_lost')

    #blocks 
    shots_blocked = get_dat(data_defense, 'blocked_shots')
    pass_blocked = get_dat(data_defense, 'blocked_passes')
    clearances = get_dat(data_defense, 'clearances')
    def_errors = get_dat(data_defense, 'errors')
    
    
    ##### possession data #####
    #touches 
    def_penarea_touches = get_dat(data_possession, 'touches_def_pen_area')
    def_third_touches = get_dat(data_possession, 'touches_def_3rd')
    mid_third_touches = get_dat(data_possession, 'touches_mid_3rd')
    att_third_touches = get_dat(data_possession, 'touches_att_3rd')
    pen_area_touches = get_dat(data_possession, 'touches_att_pen_area')

    #carries
    carry_distance = get_dat(data_possession, 'carries_distance')
    prg_carry_distance = get_dat(data_possession, 'carries_progressive_distance')
    final_third_carries = get_dat(data_possession, 'carries_into_final_third')
    pen_area_carries = get_dat(data_possession, 'carries_into_penalty_area')
    miscontrols = get_dat(data_possession, 'miscontrols')
    dispossesed = get_dat(data_possession, 'dispossessed')

    #receiving 
    pass_received = get_dat(data_possession, 'passes_received')
    prg_pass_received = get_dat(data_possession, 'progressive_passes_received')


    ###########################################################
    #creating dataframe
    df = pd.DataFrame({
        'Date': date,
        'Day': day,
        'Matchweek': matchweek,
        'Venue': venue,
        'Result': result,
        'Team': team,
        'Opponent': opponent, 
        'Start': start, 
        'Position': position,
        'Minutes Played': mins, 
        'Goals': goals, 
        'Assists': assist, 
        'Penalties Scored': pen_goals, 
        'Penalties Attempted': pen_attempted, 
        'Shots': shots, 
        'Shots on Target': sot, 
        'Yellow Cards': yellow, 
        'Red Cards': red, 
        'Touches': touches, 
        'Tackles': tackles,
        'Interceptions': interceptions, 
        'Blocks': blocks, 
        'xG': xg,
        'npxG': npxg,
        'xAG': xag, 
        'Shot Creating Actions': sca, 
        'Goal Creating Actions': gca, 
        'Passes Completed': passes_completed, 
        'Passes Attempted': passes_attempted, 
        'Progressive Passes': prg_passes,
        'Carries': carries, 
        'Progressive Carries': prg_carries, 
        'Take-ons Attempted': takeon_att, 
        'Successful Take-ons': takeon_suc,
        
        'Passing Distance': pass_distance, 
        'Progressive Passing Distance': prg_pass_distance,
        'Short Passes Completed' : short_pass_completed,
        'Short Passes Attempted': short_pass_attempted,
        'Medium Passes Completed': med_pass_completed,
        'Medium Passes Attempted': med_pass_attempted,
        'Long Passes Completed': long_pass_completed,
        'Long Passes Attempted': long_pass_attempted,
        'Expected Assists': xa,
        'Key Passes': keypass,
        'Passes into Final Third': finalthird_pass,
        'Passes into Penalty Area': penaltyarea_pass,
        'Crosses into Penalty Area': penaltyarea_cross,
    
        'Live Pass': live_pass,
        'Dead Pass': dead_pass,
        'Free Kick Pass': freekick_pass,
        'Through Balls': through_balls,
        'Switches': switches,
        'Crosses': crosses,
        'Throw Ins Taken': throwin,
        'Corners Taken': corners,
        'Passes Offside': offside_pass,
    
        'Live SCA': live_sca,
        'Deadball SCA': deadball_sca,
        'Take-on SCA': takeons_sca,
        'Shot SCA': shots_sca,
        'Foul SCA': fouls_sca,
        'Defense SCA': defense_sca,
    
        'Live GCA': live_gca,
        'Deadball GCA': deadball_gca,
        'Take-on GCA': takeons_gca,
        'Shot GCA': shots_gca,
        'Foul GCA': fouls_gca,
        'Defense GCA': defense_gca,
    
        'Tackles Won': tackles_won,
        'Defensive Third Tackles': tackles_def,
        'Middle Third Tackles': tackles_mid,
        'Attacking Third Tackles': tackles_att,
        'Dribblers Tackled': dribblers_tackled,
        'Dribblers Tackled Attempts': dribbles_challenged,
        'Challenges Lost': challenges_lost,
        'Shots Blocked': shots_blocked,
        'Passes Blocked': pass_blocked,
        'Clearances': clearances,
        'Defensive Errors': def_errors,
    
        'Defensive Penalty Area Touches': def_penarea_touches,
        'Defensive Third Touches': def_third_touches,
        'Middle Third Touches': mid_third_touches,
        'Attacking Third Touches': att_third_touches,
        'Penalty Area Touches': pen_area_touches,
        'Carry Distance': carry_distance,
        'Progressive Carry Distance': prg_carry_distance,
        'Final Third Carries': final_third_carries,
        'Carries into Penalty Area': pen_area_carries,
        'Miscontrols': miscontrols,
        'Dispossessed': dispossesed,
        'Passes Received': pass_received,
        'Progressive Passes Received': prg_pass_received
    })
    
    #modifying data types
    df = df.astype({
        'Date': 'datetime64[ns]',
        'Day': 'object',
        'Matchweek': 'int64',
        'Venue': 'object',
        'Result': 'object',
        'Team': 'object',
        'Opponent': 'object', 
        'Start': 'object', 
        'Position': 'object',
        'Minutes Played': 'int64', 
        'Goals': 'int64', 
        'Assists': 'int64', 
        'Penalties Scored': 'int64', 
        'Penalties Attempted': 'int64', 
        'Shots': 'int64', 
        'Shots on Target': 'int64', 
        'Yellow Cards': 'int64', 
        'Red Cards': 'int64', 
        'Touches': 'int64', 
        'Tackles': 'int64',
        'Interceptions': 'int64', 
        'Blocks': 'int64', 
        'xG': 'float64',
        'npxG': 'float64',
        'xAG': 'float64', 
        'Shot Creating Actions': 'int64', 
        'Goal Creating Actions': 'int64', 
        'Passes Completed': 'int64', 
        'Passes Attempted': 'int64', 
        'Progressive Passes': 'int64',
        'Carries': 'int64', 
        'Progressive Carries': 'int64', 
        'Take-ons Attempted': 'int64', 
        'Successful Take-ons': 'int64',
        
        'Short Passes Completed' : 'int64',
        'Short Passes Attempted': 'int64',
        'Medium Passes Completed': 'int64',
        'Medium Passes Attempted': 'int64',
        'Long Passes Completed': 'int64',
        'Long Passes Attempted': 'int64',
        'Expected Assists': 'float64',
        'Key Passes': 'int64',
        'Passes into Final Third': 'int64',
        'Passes into Penalty Area': 'int64',
        'Crosses into Penalty Area': 'int64',
    
        'Live Pass': 'int64',
        'Dead Pass': 'int64',
        'Free Kick Pass': 'int64',
        'Through Balls': 'int64',
        'Switches': 'int64',
        'Crosses': 'int64',
        'Throw Ins Taken': 'int64',
        'Corners Taken': 'int64',
        'Passes Offside': 'int64',
    
        'Live SCA': 'int64',
        'Deadball SCA': 'int64',
        'Take-on SCA': 'int64',
        'Shot SCA': 'int64',
        'Foul SCA': 'int64',
        'Defense SCA': 'int64',
    
        'Live GCA': 'int64',
        'Deadball GCA': 'int64',
        'Take-on GCA': 'int64',
        'Shot GCA': 'int64',
        'Foul GCA': 'int64',
        'Defense GCA': 'int64',
    
        'Tackles Won': 'int64',
        'Defensive Third Tackles': 'int64',
        'Middle Third Tackles': 'int64',
        'Attacking Third Tackles': 'int64',
        'Dribblers Tackled': 'int64',
        'Dribblers Tackled Attempts': 'int64',
        'Challenges Lost': 'int64',
        'Shots Blocked': 'int64',
        'Passes Blocked': 'int64',
        'Clearances': 'int64',
        'Defensive Errors': 'int64',
    
        'Defensive Penalty Area Touches': 'int64',
        'Defensive Third Touches': 'int64',
        'Middle Third Touches': 'int64',
        'Attacking Third Touches': 'int64',
        'Penalty Area Touches': 'int64',
        'Carry Distance': 'float64',
        'Progressive Carry Distance': 'float64',
        'Final Third Carries': 'int64',
        'Carries into Penalty Area': 'int64',
        'Miscontrols': 'int64',
        'Dispossessed': 'int64',
        'Passes Received': 'int64',
        'Progressive Passes Received': 'int64'
    })
    
    #changing date column data type 
    df['Date'] = df['Date'].dt.date
    
    ###############################################################################
    
    #now we want to get the fpl data 
    start_year = year_range[:4] #getting the whole of the first year 
    end_year = year_range[-2:]
    format_year = f'{start_year}-{end_year}'
    directory = f'Fantasy-Premier-League/data/{format_year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if fpl_name in f]
    fpldf = pd.read_csv(os.path.join(directory, csv_file[0]) + '/gw.csv')

    #some of the columns are duplicated (from the webscraped data), so we want to remove these 
    fpldf = fpldf.drop(['assists', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'fixture',
                                          'goals_conceded', 'goals_scored', 'penalties_missed', 'penalties_saved', 'red_cards'
                                          ,'team_a_score', 'team_h_score', 'was_home', 'yellow_cards', 'element', 'opponent_team', 'starts'
                                          ,'expected_goals_conceded'], axis=1, 
                       errors = 'ignore')

    #we now want to change the 'kickoff time' feature into a datetime object 
    fpldf['kickoff_time'] = pd.to_datetime(fpldf['kickoff_time'])

    #creating new column, which is just the kickoff date 
    fpldf['kickoff_date'] = fpldf['kickoff_time'].dt.date
    
    finaldf = pd.merge(df, fpldf, left_on='Date', right_on='kickoff_date', how='inner')
    
    return(finaldf)

In [ ]:
def get_url_final(code, year_range, category, player):
    base_url = 'https://fbref.com/en/players/{}/matchlogs/{}/c9/{}/{}-Match-Logs'
    url = base_url.format(code, year_range, category, player)
    html = requests.get(url).text
    soup = BeautifulSoup(html, 'lxml')
    html_filtered = soup.find('tbody')
    return(html_filtered)

In [204]:
year_list = ('2023-24', '2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18')
active_years = []
for year in year_list:
    directory = f'Fantasy-Premier-League/data/{year}/players'
    files = os.listdir(directory)
    csv_file = [f for f in files if 'Isaac_Hayden' in f]
    if csv_file:
        update_year = year[:5] + '20' + year[5:]
        active_years.append(update_year)

# dataframes = {}
# for year in active_years:
#     dataframes[year] = get_data('178ae8f8', year, 'Diogo-Jota',  "Diogo_Teixeira")
#     time.sleep(60)

active_years

['2023-2024', '2021-2022', '2020-2021', '2019-2020', '2018-2019', '2017-2018']

In [ ]:
matheusn_finaldat = compile_dat('e6af02e0', 'Matheus-Nunes', 'Matheus Luiz_Nunes', checkgames = True)

In [164]:
test = get_data('e6af02e0', '2022-2023', 'Matheus-Nunes', 'Matheus Luiz_Nunes')

ValueError: invalid literal for int() with base 10: '': Error while type casting for column 'Minutes Played'